<a href="https://colab.research.google.com/github/bumbum2403/ML_Implementation/blob/main/Bank_Statement_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Path to the uploaded CSV
csv_path = "/content/bank_statement.csv"

# Load into a DataFrame
df = pd.read_csv(csv_path)

# Normalize column names: collapse whitespace and strip
df.columns = (
    df.columns
      .astype(str)
      .str.replace(r"\s+", " ", regex=True)
      .str.replace(r"\r|\n", " ", regex=True)
      .str.strip()
)

# Remove the 'Unnamed: 6' column
if 'Unnamed: 6' in df.columns:
    df = df.drop(columns=['Unnamed: 6'])

# Display the first few rows and current columns
display(df.head())
print("\nColumns detected:", df.columns.tolist())

,Sl No,Trans_id,Value Date,Transaction Date,Transaction Posted Date,Cheque no / Ref No,Transaction Remarks,Withdrawal(Dr),Deposit(Cr),Balance
0,1.0,S3599 4460,01/May/2\n025,01/May/2025,01/05/2025\n10:23:37 AM,NaN,MMT/IMPS/5121108 02711/KAAUTOMOB ILE/KARB0000562,"25,000.00",NaN,"- 1,05,906.\n08"
1,2.0,S3690 2093,01/May/2\n025,01/May/2025,01/05/2025\n11:44:22 AM,NaN,MMT/IMPS/5121110 49866/INDIAFILTRA TION/CNRB00...,"15,000.00",NaN,"- 1,20,906.\n08"
2,3.0,S3803 5041,01/May/2\n025,01/May/2025,01/05/2025\n01:49:48 PM,NaN,MMT/IMPS/5121134 07391/BEETEE/HDF C0000330,"10,000.00",NaN,"- 1,30,906.\n08"
3,4.0,S3854 2012,01/May/2\n025,01/May/2025,01/05/2025\n02:59:45 PM,NaN,UPI/104080336349/\nUPI/kuberenterprise/ UNION ...,NaN,"5,000.00","- 1,25,906.\n08"
4,5.0,S3903 7578,01/May/2\n025,01/May/2025,01/05/2025\n03:59:17 PM,NaN,MMT/IMPS/5121157 27558/MANUPACKE RS/HDFC0000330,"5,000.00",NaN,"- 1,30,906.\n08"



Columns detected: ['Sl No', 'Trans_id', 'Value Date', 'Transaction Date', 'Transaction Posted Date', 'Cheque no / Ref No', 'Transaction Remarks', 'Withdrawal(Dr)', 'Deposit(Cr)', 'Balance']


In [ ]:
df.columns

Index(['Sl No', 'Trans_id', 'Value Date', 'Transaction Date',
       'Transaction Posted Date', 'Cheque no / Ref No', 'Transaction Remarks',
       'Withdrawal(Dr)', 'Deposit(Cr)', 'Balance'],
      dtype='object')

OLDER APPROACH UPI Regex Failed

In [ ]:
import re
import pandas as pd

# --- assume df is already loaded and cleaned as per your first cell ---

def extract_name(remark):
    text = str(remark)
    low = text.lower()

    # 1) MMT / IMPS
    if "mmt" in low or "imps" in low:
        parts = text.split("/")
        if len(parts) >= 3:
            return parts[3].strip()

    # 2) NEFT
    if "neft" in low:
        # split on '-' or '/'
        parts = re.split(r"[-/]", text)
        # find the first part after 'NEFT'
        for p in parts[2:]:
            p = p.strip()
            if p and not re.fullmatch(r"\d+", p):
                return p

    # 3) INF / INFT
    if "inf" in low:
        parts = text.split("/")
        if len(parts) >= 3:
            return parts[3].strip()

    # 4) UPI
    if "upi" in low:
        parts = text.split("/")
        # Iterate through parts to find a likely enterprise name
        for i in range(len(parts)):
            p = parts[i].strip()
            low_p = p.lower()

            # Skip empty or purely numeric parts
            if not p or re.fullmatch(r"[0-9]+", p):
                continue

            # Look for common bank names or patterns
            if "bank" in low_p or "ltd" in low_p or "enterprise" in low_p or "filters" in low_p:
                 return p

            # If it's not a common bank/enterprise name, check if the next part looks like a bank name
            if i + 1 < len(parts):
                next_p = parts[i+1].strip()
                low_next_p = next_p.lower()
                if "bank" in low_next_p or "ltd" in low_next_p:
                    return p # Return the current part if the next part is a bank name

            # As a fallback, return the first non-numeric part after 'UPI' that is not too short
            if i > 0 and not re.fullmatch(r"[0-9]+", p) and len(p) > 3:
                 return p


    # 5) Fallback: first all‑letter token ≥4 chars
    m = re.search(r"\b([A-Za-z ]{4,})\b", text)
    if m:
        return m.group(1).strip()

    # If all else fails, return the full remark
    return text.strip()

# Apply to create a new column
df["Particulars"] = df["Transaction Remarks"].apply(extract_name)

# Preview
from IPython.display import display
display(df[["Transaction Remarks", "Particulars"]].head(10))

,Transaction Remarks,Particulars
0,MMT/IMPS/5121108 02711/KAAUTOMOB ILE/KARB0000562,KAAUTOMOB ILE
1,MMT/IMPS/5121110 49866/INDIAFILTRA TION/CNRB00...,INDIAFILTRA TION
2,MMT/IMPS/5121134 07391/BEETEE/HDF C0000330,BEETEE
3,UPI/104080336349/\nUPI/kuberenterprise/ UNION ...,kuberenterprise
4,MMT/IMPS/5121157 27558/MANUPACKE RS/HDFC0000330,MANUPACKE RS
5,NEFT- UCBAN5202505010 5960242-JYOTI POWER-/FAS...,JYOTI POWER
6,NEFT- HDFCN5202505010 7389847-MAX DG POWER-000...,MAX DG POWER
7,407105500112:Int.C\noll:02-04-2025 to 01-05-2025,to
8,TRF TO FD no. 662613005141,TRF TO FD no
9,INF/INFT/040120395 031/SURYAFILTRATI ON,SURYAFILTRATI ON


basic RegEx Strategy for forming the Particular Column

In [ ]:
import re
import pandas as pd

# --- assume df is already loaded and cleaned as per your first cell ---

# 1) UPI-specific extractor
BANKS = {"axis bank", "sbi", "hdfc bank", "icici bank" ,"union bank", "Indusind"}

def extract_upi_name(remark):
    toks = [t.strip() for t in remark.split("/") if t.strip()]
    # 1. explicit UPI handle (email-like)
    for t in toks:
        if re.fullmatch(r"[\w.+-]+@[\w.-]+", t):
            return t
    # 2. 10-digit phone UPI
    for t in toks:
        if re.fullmatch(r"[6-9]\d{9}", t):
            return t
    # 3. filter out hex and banks
    cand = []
    for t in toks:
        if re.fullmatch(r"[A-Fa-f0-9]{12,}", t):
            continue
        if t.lower() in BANKS:
            continue
        cand.append(t)
    # 4. pick first reasonable token (4-20 chars, has letter)
    for t in cand:
        if 4 <= len(t) <= 20 and re.search(r"[A-Za-z]", t):
            return t
    # 5. fallback to first candidate
    if cand:
        return cand[0]
    # 6. ultimate fallback
    return remark.strip()

# 2) General extractor with UPI hook
def extract_name(remark):
    text = str(remark).strip()
    low = text.lower()

    # MMT / IMPS
    if "mmt" in low or "imps" in low:
        parts = text.split("/")
        if len(parts) >= 4:
            return parts[3].strip()

    # NEFT
    if "neft" in low:
        parts = re.split(r"[-/]", text)
        for p in parts[2:]:
            p = p.strip()
            if p and not re.fullmatch(r"\d+", p):
                return p

    # INF / INFT
    if "inf" in low:
        parts = text.split("/")
        if len(parts) >= 4:
            return parts[3].strip()

    # UPI
    if "upi" in low:
        return extract_upi_name(text)

    # Fallback: first all-letter token ≥4 chars
    m = re.search(r"\b([A-Za-z ]{4,})\b", text)
    if m:
        return m.group(1).strip()

    return text

# 3) Apply to DataFrame
df["Particulars"] = df["Transaction Remarks"].apply(extract_name)

# 4) Preview results
from IPython.display import display
display(df[["Transaction Remarks", "Particulars"]].head(10))


,Transaction Remarks,Particulars
0,MMT/IMPS/5121108 02711/KAAUTOMOB ILE/KARB0000562,KAAUTOMOB ILE
1,MMT/IMPS/5121110 49866/INDIAFILTRA TION/CNRB00...,INDIAFILTRA TION
2,MMT/IMPS/5121134 07391/BEETEE/HDF C0000330,BEETEE
3,UPI/104080336349/\nUPI/kuberenterprise/ UNION ...,kuberenterprise
4,MMT/IMPS/5121157 27558/MANUPACKE RS/HDFC0000330,MANUPACKE RS
5,NEFT- UCBAN5202505010 5960242-JYOTI POWER-/FAS...,JYOTI POWER
6,NEFT- HDFCN5202505010 7389847-MAX DG POWER-000...,MAX DG POWER
7,407105500112:Int.C\noll:02-04-2025 to 01-05-2025,to
8,TRF TO FD no. 662613005141,TRF TO FD no
9,INF/INFT/040120395 031/SURYAFILTRATI ON,SURYAFILTRATI ON


## FEATURE TRANSFORMATION FOR PARTICULARS COLUMN:
-- Advance RegEx Strategies


In [38]:
import re
import pandas as pd

# 1) UPI-specific extractor
# Expanded list of bank names (case-insensitive) for filtering
BANKS = {
    "axis bank", "sbi", "hdfc bank", "icici bank", "union bank", "indusind",
    "bandhan bank lt", "canara bank", "yes bank limite", "central bank of",
    "punjab national", "state bank of i", "state bank of india" # Added based on examples
}

# Expanded list of generic phrases that are unlikely to be useful "particulars"
GENERIC_PHRASES = {
    "payment from ph", "na", "bank account xx", "upi", "bill no",
    "payment on cred", "payment from phone", "transfer", "neft", "imps", "mmt", "inf", "inft"
}

def extract_upi_name(remark):
    """
    Extracts the most relevant 'particular' (merchant/recipient name)
    from a UPI transaction remark, building upon the original logic.

    Args:
        remark (str): The raw transaction remark string.

    Returns:
        str: The extracted particular, or the original cleaned remark as a fallback.
    """
    # Step 0: Clean the remark string - replace newlines with spaces and normalize multiple spaces
    remark = str(remark).replace('\n', ' ').strip()
    remark = re.sub(r'\s+', ' ', remark)

    # Split the remark into tokens based on '/' delimiter
    toks = [t.strip() for t in remark.split("/") if t.strip()]

    # Priority 1: Explicit UPI handle (email-like format: name@bank)
    # This regex is improved to allow spaces within the username part (e.g., "paytm- 537704@pt")
    # Group 1: Username part (before @), Group 2: Domain part (after @)
    upi_handle_regex = re.compile(r"([\w.+-]+(?:\s[\w.+-]+)*)@([\w.-]+)", re.IGNORECASE)

    for t in toks:
        match = upi_handle_regex.fullmatch(t)
        if match:
            full_handle = t # Keep the original matched token for fallback if needed
            name_part = match.group(1).strip() # The part before '@'

            # Specific mapping for common UPI app prefixes to cleaner names
            low_name_part = name_part.lower()
            if low_name_part.startswith("paytm-"):
                return "Paytm"
            if low_name_part.startswith("phonepe-"):
                return "PhonePe"
            if low_name_part.startswith("googlepay-") or low_name_part.startswith("gpay-"):
                return "Google Pay"
            if low_name_part.startswith("cred."):
                return "CRED"

            # If the name part is purely numeric (like a mobile number), return the full handle.
            # This handles cases like "9845355791@ib" where the number itself is the identifier.
            if re.fullmatch(r"\d+", name_part.replace(" ", "")): # Check numeric after removing spaces
                return full_handle

            # Otherwise, return the cleaned name part (e.g., "jain.balchand1" from "jain.balchand1@")
            return name_part

    # Priority 2: 10-digit phone number as a standalone token
    for t in toks:
        if re.fullmatch(r"[6-9]\d{9}", t):
            return t

    # Priority 3: Filtered Candidates (General Merchant Names)
    # Build a list of potential candidates after filtering out known noise
    cand = []
    for t in toks:
        low_t = t.lower()

        # Filter out known generic phrases and bank names
        if low_t in BANKS or low_t in GENERIC_PHRASES:
            continue

        # Filter out long numeric strings (likely Transaction Reference Numbers - RRNs)
        if re.fullmatch(r"\d{10,}", t): # 10 or more digits
            continue

        # Filter out very long alphanumeric strings (likely UPI Transaction IDs or hashes)
        # These often contain a mix of letters and numbers and are typically > 12 chars.
        if re.fullmatch(r"[A-Fa-f0-9]{12,}", t): # Original hex filter
            continue
        if re.fullmatch(r"[A-Za-z0-9]{15,}", t) and not re.search(r"[A-Za-z]{3,}", t): # Longer alphanumeric, but not clearly a name
            continue

        # Filter out tokens that are too short and don't contain letters (e.g., single digits, symbols)
        if len(t) < 3 and not re.search(r"[A-Za-z]", t):
            continue

        cand.append(t)

    # Priority 4: Pick the first "reasonable" token from the filtered candidates
    # A "reasonable" token is typically 4-40 characters long and contains at least one letter.
    # This is the heuristic that worked well for my original examples.
    for t in cand:
        if 4 <= len(t) <= 40 and re.search(r"[A-Za-z]", t):
            return t.strip()

    # Priority 5: Fallback to the first remaining candidate if no "reasonable" one is found
    if cand:
        return cand[0].strip()

    # Ultimate fallback: return the original cleaned remark if nothing else fits
    return remark.strip()

# 2) General extractor with UPI hook (no changes needed here, as extract_upi_name is improved)
def extract_name(remark):
    """
    Extracts a general 'particular' based on transaction type (MMT/IMPS, NEFT, INF, UPI).
    """
    text = str(remark).strip()
    low = text.lower()

    # MMT / IMPS
    if "mmt" in low or "imps" in low:
        parts = text.split("/")
        if len(parts) >= 4:
            return parts[3].strip()

    # NEFT
    if "neft" in low:
        # Split by both '-' and '/' for NEFT remarks
        parts = re.split(r"[-/]", text)
        for p in parts[2:]: # Start checking from the third part
            p = p.strip()
            if p and not re.fullmatch(r"\d+", p): # Return first non-numeric part
                return p

    # INF / INFT (Assuming similar structure to IMPS/NEFT, often for internal funds transfer)
    if "inf" in low:
        parts = text.split("/")
        if len(parts) >= 4:
            return parts[3].strip()

    # UPI - Delegate to the specialized UPI extractor
    if "upi" in low:
        return extract_upi_name(text)

    # Fallback: first all-letter token ≥4 chars if no specific type is matched
    m = re.search(r"\b([A-Za-z ]{4,})\b", text)
    if m:
        return m.group(1).strip()

    # Ultimate fallback: return the original text if no pattern matches
    return text

# 3) Apply to DataFrame
# This line will be executed by your existing notebook cell
df["Particulars"] = df["Transaction Remarks"].apply(extract_name)

# 4) Preview results
# This block will be executed by your existing notebook cell
from IPython.display import display
display(df[["Transaction Remarks", "Particulars"]].head(10))


,Transaction Remarks,Particulars
0,MMT/IMPS/5121108 02711/KAAUTOMOB ILE/KARB0000562,KAAUTOMOB ILE
1,MMT/IMPS/5121110 49866/INDIAFILTRA TION/CNRB00...,INDIAFILTRA TION
2,MMT/IMPS/5121134 07391/BEETEE/HDF C0000330,BEETEE
3,UPI/104080336349/\nUPI/kuberenterprise/ UNION ...,kuberenterprise
4,MMT/IMPS/5121157 27558/MANUPACKE RS/HDFC0000330,MANUPACKE RS
5,NEFT- UCBAN5202505010 5960242-JYOTI POWER-/FAS...,JYOTI POWER
6,NEFT- HDFCN5202505010 7389847-MAX DG POWER-000...,MAX DG POWER
7,407105500112:Int.C\noll:02-04-2025 to 01-05-2025,to
8,TRF TO FD no. 662613005141,TRF TO FD no
9,INF/INFT/040120395 031/SURYAFILTRATI ON,SURYAFILTRATI ON


In [ ]:
df[["Transaction Remarks", "Particulars"]]

,Transaction Remarks,Particulars
0,MMT/IMPS/5121108 02711/KAAUTOMOB ILE/KARB0000562,KAAUTOMOB ILE
1,MMT/IMPS/5121110 49866/INDIAFILTRA TION/CNRB00...,INDIAFILTRA TION
2,MMT/IMPS/5121134 07391/BEETEE/HDF C0000330,BEETEE
3,UPI/104080336349/\nUPI/kuberenterprise/ UNION ...,kuberenterprise
4,MMT/IMPS/5121157 27558/MANUPACKE RS/HDFC0000330,MANUPACKE RS
...,...,...
225,UPI/515103385856/\nUPI/kuberenterprise/ UNION ...,kuberenterprise
226,INF/INFT/040436112 201/STUFILTER,STUFILTER
227,MMT/IMPS/5151187 84559/SWATIRATHI\n/PUNB0115510,SWATIRATHI
228,NEFT- HDFCN5202505316 5169481-YASH GEN POWER S...,YASH GEN POWER SOLUTIONS


## view by groups

types of payment groups, for example:

1. MMT / IMPS

2. NEFT

3. UPI

4. Proceed Closure

5. FD

6. CLG

In [39]:
pd.set_option('display.max_colwidth', None)



# Filter the DataFrame for rows where 'Transaction Remarks' contains "UPI"
upi_transactions = df[df['Transaction Remarks'].str.contains('UPI', na=False)]

# Display SHAPE/ROWS of filtered DataFrame

print("Shape of selcted table: ", upi_transactions.shape)
print("Total number of transactions within selected group: ",len(upi_transactions))

# Display the filtered rows
display(upi_transactions[["Transaction Remarks", "Particulars"]])


pd.reset_option('display.max_colwidth')


Shape of selcted table:  (48, 11)
Total number of transactions within selected group:  48


,Transaction Remarks,Particulars
3,UPI/104080336349/\nUPI/kuberenterprise/ UNION BANK OF\nI/HDFf8623abfc1c74 e23b56fb9b8979ce1 ab,kuberenterprise
14,UPI/maasheetlaenter\n/UPI/BANDHAN BANK LT/512340527074/A XI37591a7f555b4cc\n59de0bb36ad416a8 5,maasheetlaenter
17,UPI/paytm- 537704@pt/Paymen t from Ph/YES BANK LIMITE/7539807809\n19/IBL99be9aa9f33 847af8becd85691d3 3ddb,Paytm
19,UPI/maasheetlaenter\n/UPI/BANDHAN BANK LT/512368247683/A XI971492152af841a\nba70bcda1ca5a9b0 4,maasheetlaenter
20,UPI/namitadieselspa\n/bill no 973/State Bank Of I/512381480244/SBI\ncdc37c93f42746e19 f3fb4fe4b0c4ae0,namitadieselspa
39,UPI/maasheetlaenter\n/UPI/BANDHAN BANK LT/549303723274/A XI3c8b4c067b64416\n4a23b66bf81d2d2e 0,maasheetlaenter
40,UPI/maasheetlaenter\n/UPI/BANDHAN BANK LT/549316514413/A XI8fb6cba24860416\n390f41eda959bc692,maasheetlaenter
48,UPI/104423332523/\nUPI/kuberenterprise/ UNION BANK OF\nI/HDFc30ca3f40bfd4 517985e269d67af38\n6f,kuberenterprise
75,UPI/MAB.037214012\n63/Payment from Ph/AXIS BANK/78455478355 5/IBLe36d0d12a563\n4d67bd607b573da7 2350/,MAB.037214012 63
77,UPI/paytm.s15h3s6 @p/Payment from Ph/YES BANK PTY/363114347367/ IBL271f881f42c24cf 69e8000e537790b3\n2/,paytm.s15h3s6 @p


## ROW TRACKER SCRIPT

In [ ]:
pd.set_option('display.max_colwidth', None)

# Replace row_number with the actual row number you want to view
row_number = 103
# Get the entire row using iloc
entire_row = df.iloc[row_number]

# Display the entire row
display(entire_row)

# Reset pandas option to default
pd.reset_option('display.max_colwidth')

,103
Sl No,103.0
Trans_id,S7212 6907
Value Date,15/May/2\n025
Transaction Date,15/May/2025
Transaction Posted Date,15/05/2025\n07:39:12 AM
Cheque no / Ref No,NaN
Transaction Remarks,UPI/Bank Account XX/Payment from Ph/CENTRAL BANK OF/460346377015/I\nBL920c44d7ee8a43 7994817380d23356\n5f
Withdrawal(Dr),"20,000.00"
Deposit(Cr),NaN
Balance,"- 1,20,678.\n08"


## START BUILDING SEPERATE TABLES

In [43]:
df.shape

(230, 11)

In [45]:
import numpy as np

# Ensure 'Transaction Date' is datetime for consistency, if not already
# Using errors='coerce' will turn unparseable dates into NaT (Not a Time)
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

# Clean and convert 'Withdrawal(Dr)' and 'Deposit(Cr)' to numeric
# Remove commas and newlines, then convert to float
for col in ['Withdrawal(Dr)', 'Deposit(Cr)']:
    if col in df.columns:
        # Ensure the column is treated as string before applying string methods
        df[col] = df[col].astype(str).str.replace(r'[,\n]', '', regex=True)
        # Convert to numeric, coercing errors will turn invalid parsing into NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')

# --- Create DEBIT Table ---
# Select rows where 'Withdrawal(Dr)' has a non-null and non-zero value
# Now that the column is numeric, the comparison > 0 is valid
debit_df = df[df['Withdrawal(Dr)'].notna() & (df['Withdrawal(Dr)'] > 0)].copy()

# Select and rename columns for the Debit Table
debit_df = debit_df.loc[:, [
    'Transaction Date',
    'Particulars',
    'Cheque no / Ref No',
    'Withdrawal(Dr)'
]]
debit_df.columns = ['Date', 'Particular', 'Cheq_no.', 'Amount']

# Add the empty 'L/F (null)' column
debit_df['L/F (null)'] = np.nan # Or None, np.nan is generally preferred for numeric columns

# Reorder columns to match the desired output: <Date, Particular, Cheq_no. , L/F(null) , Amount>
debit_df = debit_df[['Date', 'Particular', 'Cheq_no.', 'L/F (null)', 'Amount']]


# --- Create CREDIT Table ---
# Select rows where 'Deposit(Cr)' has a non-null and non-zero value
# Now that the column is numeric, the comparison > 0 is valid
credit_df = df[df['Deposit(Cr)'].notna() & (df['Deposit(Cr)'] > 0)].copy()

# Select and rename columns for the Credit Table
credit_df = credit_df.loc[:, [
    'Transaction Date',
    'Particulars',
    'Cheque no / Ref No',
    'Deposit(Cr)'
]]
credit_df.columns = ['Date', 'Particular', 'Cheq_no.', 'Amount']

# Add the empty 'L/F (null)' column
credit_df['L/F (null)'] = np.nan

# Reorder columns to match the desired output
credit_df = credit_df[['Date', 'Particular', 'Cheq_no.', 'L/F (null)', 'Amount']]

# --- Display the first few rows of the new DataFrames ---
print("--- DEBIT TABLE ---")
from IPython.display import display
display(debit_df.head())

print("\n--- CREDIT TABLE ---")
display(credit_df.head())

--- DEBIT TABLE ---


,Date,Particular,Cheq_no.,L/F (null),Amount
0,2025-05-01,KAAUTOMOB ILE,NaN,NaN,25000.0
1,2025-05-01,INDIAFILTRA TION,NaN,NaN,15000.0
2,2025-05-01,BEETEE,NaN,NaN,10000.0
4,2025-05-01,MANUPACKE RS,NaN,NaN,5000.0
7,2025-05-02,to,NaN,NaN,1236.0



--- CREDIT TABLE ---


,Date,Particular,Cheq_no.,L/F (null),Amount
3,2025-05-01,kuberenterprise,NaN,NaN,5000.0
5,2025-05-01,JYOTI POWER,NaN,NaN,20000.0
6,2025-05-01,MAX DG POWER,NaN,NaN,50000.0
10,2025-05-02,ANEES ENTERPRISES,NaN,NaN,40000.0
11,2025-05-02,RAJDHANI SALES CORPORAT,NaN,NaN,10000.0


In [46]:
# You can also check their info to confirm data types and non-null counts
print("\nDebit Table Info:")
debit_df.info()
print("\nCredit Table Info:")
credit_df.info()


Debit Table Info:
<class 'pandas.core.frame.DataFrame'>
Index: 105 entries, 0 to 229
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        105 non-null    datetime64[ns]
 1   Particular  105 non-null    object        
 2   Cheq_no.    6 non-null      float64       
 3   L/F (null)  0 non-null      float64       
 4   Amount      105 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 4.9+ KB

Credit Table Info:
<class 'pandas.core.frame.DataFrame'>
Index: 124 entries, 3 to 228
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        124 non-null    datetime64[ns]
 1   Particular  124 non-null    object        
 2   Cheq_no.    0 non-null      float64       
 3   L/F (null)  0 non-null      float64       
 4   Amount      124 non-null    float64       
dtypes: datetime64[n

In [52]:
credit_df[["Date", "Particular", "Amount"]]
debit_df[["Date", "Particular", "Amount"]]

# df[["Transaction Remarks", "Particulars"]]

,Date,Particular,Amount
0,2025-05-01,KAAUTOMOB ILE,25000.0
1,2025-05-01,INDIAFILTRA TION,15000.0
2,2025-05-01,BEETEE,10000.0
4,2025-05-01,MANUPACKE RS,5000.0
7,2025-05-02,to,1236.0
...,...,...,...
221,2025-05-30,GAURAV,25000.0
223,2025-05-31,CASH PAID,45000.0
226,2025-05-31,STUFILTER,4130.0
227,2025-05-31,SWATIRATHI,14000.0
